In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Dataset

In [ ]:
import pandas as pd

data_path = "/content/drive/MyDrive/Datasets/Processed_NewsData/NewsData2022_2023-06-15.csv"
merged = pd.read_csv(data_path, encoding="utf-8", lineterminator='\n')
# merged = pd.read_csv("/content/drive/MyDrive/Datasets/TestEventExtractionDataWithSentiments.csv", encoding="utf-8", lineterminator='\n')
for column in merged.columns:
  if "Unnamed" in column:
    del merged[column]
  elif "index" in column:
    del merged[column]
# merged['DateTime'] = pd.to_datetime(merged['DateTime'])
# merged = merged.sort_values('DateTime')
# merged.reset_index(drop=True, inplace=True)
print(merged.columns)
print(merged.shape)
merged.head(2)

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment', 'CC_tokenized', 'NER_Persons', 'NER_Orgs', 'NER_Loc'],
      dtype='object')
(103169, 20)


,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized,NER_Persons,NER_Orgs,NER_Loc
0,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا؟,2017-08-05,https://www.bbc.com/urdu/regional-40825452,یہ تحریر پہلی مرتبہ بی بی سی اردو کی ویب سائٹ ...,BBC,پاکستان,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا؟ یہ ...,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا یہ ت...,مہندرا اینڈ محمد تقسیم الگ تحریر مرتبہ بی بی ا...,مہندرا اینڈ محمد تقسیم الگ کیا؟ تحریر مرتبہ بی...,مہندرا اینڈ محمد تقسیم الگ تحریر مرتبہ بی بی ا...,pakistan,NotAssigned,"['مہندرا', 'کیشو', 'غلام', 'محمد', 'اینڈ', 'ان...",2345,1.0,"['مہندرا', 'اینڈ', 'محمد', 'کو', 'تقسیم', 'نے'...","['مہندرا', 'قارئین', 'بیٹن', 'چبھن', 'محمد', '...",['مسلم'],"['پاکستان', 'والا']"
1,تقسیم کی ایک نامکمل محبت کی کہانی,2017-08-15,https://www.bbc.com/urdu/regional-40912412,یہ تحریر پہلی مرتبہ 15 اگست سنہ 2017 کو بی بی ...,BBC,پاکستان,تقسیم کی ایک نامکمل محبت کی کہانی یہ تحریر پہل...,تقسیم کی ایک نامکمل محبت کی کہانی یہ تحریر پہل...,تقسیم نامکمل محبت کہانی تحریر مرتبہ 15 اگست سن...,تقسیم نامکمل محبت کہانی تحریر مرتبہ 15 اگست سن...,تقسیم نامکمل محبت کہانی تحریر مرتبہ 15 اگست سن...,pakistan,NotAssigned,"['عصمت', 'جیتو', 'کملا', 'پٹیل', 'عورتوں', 'ام...",2372,1.0,"['تقسیم', 'کی', 'ایک', 'نامکمل', 'محبت', 'کی',...","['میموائر', 'برباد', 'عصمت انھی', 'انھی', 'مین...",[],"['پاکستان', 'لاہور', 'ہندوستان', 'راولپنڈی', '..."


# Load RoBERTa

In [ ]:
! pip install transformers
! pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
# Load Urduhack/roberta tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")
model = AutoModel.from_pretrained("urduhack/roberta-urdu-small")

# Example usage: tokenize and encode a sentence
# sentence = "آپ کیسے ہیں؟"
# tokens1 = tokenizer2.tokenize(sentence, padding=True, truncation=True, return_tensors='tf')
# input_ids1 = tokenizer2.encode(sentence, add_special_tokens=True)

In [ ]:
import torch
def get_roberta_embeddings(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embeddings

# Creating Subsets for Large Datasets

In [ ]:
# Convert the 'DateTime' column to a datetime object if it's not already
merged['DateTime'] = pd.to_datetime(merged['DateTime'])

# Group the data by month
grouped = merged.groupby(merged['DateTime'].dt.month)

# Create a dictionary to store the subsets
subsets = {}

# Iterate over the groups and store each subset
for month, group in grouped:
    subsets[month] = group

# Print the shape of each subset
for month, subset in subsets.items():
    print(f"Subset for month {month} shape:", subset.shape)


Subset for month 1 shape: (7324, 20)
Subset for month 2 shape: (8633, 20)
Subset for month 3 shape: (9651, 20)
Subset for month 4 shape: (9545, 20)
Subset for month 5 shape: (12475, 20)
Subset for month 6 shape: (11427, 20)
Subset for month 7 shape: (9882, 20)
Subset for month 8 shape: (8273, 20)
Subset for month 9 shape: (6457, 20)
Subset for month 10 shape: (6272, 20)
Subset for month 11 shape: (6515, 20)
Subset for month 12 shape: (6715, 20)


In [ ]:
# month = subsets[4]
m = 12
month = subsets[m]
month.shape

(6715, 20)

# Get News Embeddings

## Without Dividing into Subsets

In [ ]:
import numpy as np
merged['Embedding'] = ''
merged['Embedding_Vector'] = ''
for index, row in merged.iterrows():
  article = row['CC_wo_Stopwords_punc']
  vectors = get_roberta_embeddings(article)
  query_embedding = np.mean(vectors, axis=0)
  # print(vectors)
  # print(query_embedding)
  # print(type(query_embedding))
  merged['Embedding_Vector'][index] = vectors
  merged.loc[index, 'Embedding'] = query_embedding

<ipython-input-6-e53e80cc8b7d>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['Embedding_Vector'][index] = vectors


In [ ]:
print(f"Vector Shape: {vectors.shape}")
print(f"Vector Type: {type(vectors)}")
print(f"Dataset Columns: {merged.columns}")

In [ ]:
merged.head(2)

In [ ]:
merged.to_csv(data_path, index=False)

## With Subsets

In [ ]:
%%time
import numpy as np
month['Embedding'] = ''
month['Embedding_Vector'] = ''
for index, row in month.iterrows():
  article = row['CC_wo_Stopwords_punc']
  vectors = get_roberta_embeddings(article)
  query_embedding = np.mean(vectors, axis=0)
  # print(vectors)
  # print(query_embedding)
  # print(type(query_embedding))
  month['Embedding_Vector'][index] = vectors
  month.loc[index, 'Embedding'] = query_embedding

<timed exec>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CPU times: user 1h 1min 4s, sys: 2.22 s, total: 1h 1min 6s
Wall time: 1h 1min 27s


In [ ]:
print(f"Vector Shape: {vectors.shape}")
print(f"Vector Type: {type(vectors)}")
print(f"Dataset Columns: {merged.columns}")
print(f"Dataset Columns: {month.columns}")

Vector Shape: (768,)
Vector Type: <class 'numpy.ndarray'>
Dataset Columns: Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment', 'CC_tokenized', 'NER_Persons', 'NER_Orgs', 'NER_Loc'],
      dtype='object')
Dataset Columns: Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment', 'CC_tokenized', 'NER_Persons', 'NER_Orgs', 'NER_Loc',
       'Embedding', 'Embedding_Vector'],
      dtype='object')


In [ ]:
month.head(2)

,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,...,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized,NER_Persons,NER_Orgs,NER_Loc,Embedding,Embedding_Vector
2,پاکستانی تاریخ کی ’تاریک ترین‘ رات جب ’ملک بچا...,2018-10-10,https://www.bbc.com/urdu/pakistan-45805505,سات اکتوبر 1958 کو لگائے جانے والے پاکستان کے ...,BBC,پاکستان,پاکستانی تاریخ کی ’تاریک ترین‘ رات جب ’ملک بچا...,پاکستانی تاریخ کی تاریک ترین رات جب ملک بچانے ...,پاکستانی تاریخ تاریک ترین رات ملک بچانے مارشل ...,پاکستانی تاریخ ’تاریک ترین‘ رات ’ملک بچانے‘ ما...,...,NotAssigned,"['اسکندر', 'مرزا', 'مارشل', 'ایوب', 'لا', 'سہر...",2054,1.0,"['پاکستانی', 'تاریخ', 'کی', '’تاریک', 'ترین', ...","['سکندر', 'اعظم', 'تاش', 'ہمایوں', 'ناہید', 'خ...","['کالعدم', 'اقبال']","['ہندوستان', 'کوڑا', 'کراچی', 'کوئٹہ', 'برطانی...",0.000289,"[0.8170644, 0.011714853, -0.33213586, -0.93070..."
12219,عوام کیلئے بری خبر، حکومت نے پیٹرول 4 روپے ليٹ...,2021-10-01,https://www.nawaiwaqt.com.pk/01-Oct-2021/1425418,کراچی: حکومت کی جانب سے رواں ماہ میں پیٹرولیم ...,Nawaiwaqt,قومی,عوام کیلئے بری خبر، حکومت نے پیٹرول 4 روپے ليٹ...,عوام کیلئے بری خبر حکومت نے پیٹرول 4 روپے لیٹر...,عوام کیلئے بری خبر حکومت پیٹرول 4 روپے لیٹر مہ...,عوام کیلئے بری خبر، حکومت پیٹرول 4 روپے ليٹر م...,...,NotAssigned,"['لیٹر', 'ڈیزل', 'پیٹرولیم', 'روپے', 'قیمت', '...",88909,1.0,"['عوام', 'کیلئے', 'بری', 'خبر', '،', 'حکومت', ...",[],[],['کراچی'],0.000305,"[0.20231852, -0.07226098, -0.43643257, -0.8594..."


In [ ]:
path = f"/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub{m}.csv"
month.to_csv(path, index=False)

# Merge All Subsets

In [ ]:
d1 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub1.csv", encoding="utf-8", lineterminator='\n')
d2 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub2.csv", encoding="utf-8", lineterminator='\n')
d3 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub3.csv", encoding="utf-8", lineterminator='\n')
d4 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub4.csv", encoding="utf-8", lineterminator='\n')
d5 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub5.csv", encoding="utf-8", lineterminator='\n')
d6 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub6.csv", encoding="utf-8", lineterminator='\n')
d7 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub7.csv", encoding="utf-8", lineterminator='\n')
d8 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub8.csv", encoding="utf-8", lineterminator='\n')
d9 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub9.csv", encoding="utf-8", lineterminator='\n')
d10 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub10.csv", encoding="utf-8", lineterminator='\n')
d11 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub11.csv", encoding="utf-8", lineterminator='\n')
d12 = pd.read_csv("/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/embedding_sub12.csv", encoding="utf-8", lineterminator='\n')

In [ ]:
merged = pd.concat([d1,d2,d3,d4,d5,d6,d7,d8,d9,d10,d11,d12,d13])
if "Unnamed: 0" in merged.columns:
  del merged['Unnamed: 0']
elif "index" in merged.columns:
  del merged['index']

merged['DateTime'] = pd.to_datetime(merged['DateTime'])
merged = merged.sort_values('DateTime')
merged.reset_index(drop=True, inplace=True)
print(merged.shape)
merged.to_csv("/content/drive/MyDrive/Datasets/Processed_News/NewsData2022_2023-06-15.csv", index=False)
merged.head(2)